Play with accessing DuckDB and Azure Storage Blobs using PyPRQL Jupyter magics

In [ ]:
%load_ext sql
%load_ext pyprql.magic

In [2]:
import utils
from azure.identity import ChainedTokenCredential, ManagedIdentityCredential, InteractiveBrowserCredential

In [41]:
tenant_id = "3a2d9a5b-25c0-47da-9dff-842d10755773"
account_name = "azduckdb"
container_name = "data"
blob_path = "HuggingFaceFW/fineweb/CC-MAIN-2013-20/*.parquet"

In [4]:
credential = ChainedTokenCredential(ManagedIdentityCredential(), InteractiveBrowserCredential(tenant_id=tenant_id))
connection_string = utils.get_connection_string(credential, account_name, container_name)

In [7]:
%prql duckdb:///:memory:

In [9]:
%%sql CREATE OR REPLACE SECRET {{account_name}} (
    TYPE AZURE,
    CONNECTION_STRING '{{connection_string}}',
    SCOPE 'az://{{account_name}}.blob.core.windows.net/{{container_name}}/'
)

Running query in 'duckdb:///:memory:'

Success


In [18]:
%sql FROM duckdb_secrets()

Running query in 'duckdb:///:memory:'

name,type,provider,persistent,storage,scope,secret_string
azduckdb,azure,config,False,memory,['az://azduckdb.blob.core.windows.net/data/'],name=azduckdb;type=azure;provider=config;serializable=true;scope=az://azduckdb.blob.core.windows.net/data/;connection_string=redacted


In [64]:
%%prql
from s"SELECT * FROM 'az://azduckdb.blob.core.windows.net/data/HuggingFaceFW/fineweb/CC-MAIN-2013-20/*.parquet'"
take 5

,text,id,dump,url,date,file_path,language,language_score,token_count
0,How AP reported in all formats from tornado-st...,<urn:uuid:d66bc6fe-8477-4adf-b430-f6a558ccc8ff>,CC-MAIN-2013-20,http://%20jwashington@ap.org/Content/Press-Rel...,2013-05-18T05:48:54Z,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.972142,717
1,"Did you know you have two little yellow, nine-...",<urn:uuid:803e14c3-dc2e-43d6-b75d-6fb3981c4fe6>,CC-MAIN-2013-20,http://1000awesomethings.com/2012/09/24/934-ad...,2013-05-18T08:11:45Z,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.947991,821
2,Car Wash For Clara!\nNow is your chance to hel...,<urn:uuid:ac1bbfff-9519-4967-9c64-3dc3a4b471ec>,CC-MAIN-2013-20,http://1027kord.com/car-wash-for-clara/,2013-05-18T06:49:55Z,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.911518,125
3,Listeners Get Sky-high View of Missoula From H...,<urn:uuid:c1445c58-b111-4c4e-badd-1e43ec317df7>,CC-MAIN-2013-20,http://1075zoofm.com/listeners-get-sky-high-vi...,2013-05-18T06:25:20Z,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.956516,103
4,Log In Please enter your ECode to log in.\nFor...,<urn:uuid:e5829f7d-b944-4468-9573-61b7cb3078cc>,CC-MAIN-2013-20,http://1105govinfoevents.com/enterprisearchite...,2013-05-18T05:27:01Z,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.798235,75


In [66]:
%%prql
from s"SELECT * FROM 'az://azduckdb.blob.core.windows.net/data/HuggingFaceFW/fineweb/CC-MAIN-2013-20/*.parquet'"
aggregate { count id }

,count_star()
0,5433000
